### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
#!pip install pandas matplotlib seaborn gensim plotly
#!pip install --upgrade numpy gensim
#!pip install --upgrade nbformat
#!pip install tensorflow
#!pip install pymupdf

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import fitz

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset el libro de Romeo y Julieta.

In [3]:
# Función para leer el contenido de un archivo .pdf desde una página de inicio hasta una página de finalización
def read_pdf(file_path, start_page, end_page):
    doc = fitz.open(file_path)
    content = []
    
    # Asegurarnos de que las páginas estén dentro de los límites válidos
    start_page = max(start_page, 0)  # Página mínima: 0 (primera página)
    end_page = min(end_page, doc.page_count - 1)  # Página máxima: la última página
    
    for page_num in range(start_page, end_page + 1):
        page = doc.load_page(page_num)  # Cargar la página
        content.append(page.get_text())  # Obtener el texto de la página
    
    return "\n".join(content)


In [4]:
# Leer el archivo .pdf desde una página de inicio hasta una página de finalización
file_path = 'William_Shakespeare_-_Romeo_y_Julieta.pdf'
start_page = 5  # Página de inicio
end_page = 132   # Página de finalización

text = read_pdf(file_path, start_page, end_page)

# Dividir el texto en líneas y crear un DataFrame
lines = text.split('\n')
df = pd.DataFrame(lines, columns=["Texto"])

# Mostrar las primeras filas del DataFrame
print(df.head())

                  Texto
0               ACTO I.
1                     6
2                      
3       ESCENA PRIMERA.
4  Una plaza de Verona.


In [5]:
# Filtrar registros vacíos
df = df[df['Texto'].str.strip().ne('')]

# Filtrar registros que son solo números
df = df[df['Texto'].str.isdigit() == False]

# Filtrar registros que tienen solo una palabra (sin espacios)
df = df[df['Texto'].str.split().str.len() > 1]

# Eliminar el carácter "º" de todos los registros
df['Texto'] = df['Texto'].str.replace('º', '', regex=False)

# Reseteo el índice
df = df.reset_index(drop=True)

# Mostrar las primeras filas del DataFrame
print(df.head())

                                               Texto
0                                            ACTO I.
1                                    ESCENA PRIMERA.
2                               Una plaza de Verona.
3        SANSON y GREGORIO, con espadas y broqueles.
4  A fe mia, Gregorio, que no hay por qué bajar l...


In [6]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 2095


### 1 - Preprocesamiento

In [29]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0]))

/tmp/ipykernel_8939/793277113.py:7: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [8]:
# Demos un vistazo
sentence_tokens[:4]

[['acto', 'i'],
 ['escena', 'primera'],
 ['una', 'plaza', 'de', 'verona'],
 ['sanson', 'y', 'gregorio', 'con', 'espadas', 'y', 'broqueles']]

### 2 - Crear los vectores (word2vec)

In [9]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [10]:
# Crearmos el modelo generador de vectores
w2v_model = Word2Vec(min_count=5,       # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,          # cant de palabras antes y desp de la predicha
                     #vector_size=300,   # dimensionalidad de los vectores 
                     vector_size=10,   # dimensionalidad de los vectores 
                     negative=20,       # cantidad de negative samples... 0 es no se usa
                     workers=1,         # si tienen más cores pueden cambiar este valor
                     sg=1)              # modelo 0:CBOW  1:skipgram


# Acá le bajé la dimensionalidad a 10 (mayor a la raíz 4ta de las palabras del corpus) y no tuve cambios contra el caso de 300

In [11]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [12]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 2095


In [13]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 552


### 3 - Entrenar embeddings

In [14]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=20,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 142144.0
Loss after epoch 1: 77909.859375
Loss after epoch 2: 77676.515625
Loss after epoch 3: 76870.65625
Loss after epoch 4: 76063.46875
Loss after epoch 5: 77645.9375
Loss after epoch 6: 77674.0
Loss after epoch 7: 77691.1875
Loss after epoch 8: 77378.3125
Loss after epoch 9: 77729.375
Loss after epoch 10: 77351.3125
Loss after epoch 11: 76879.0
Loss after epoch 12: 75370.625
Loss after epoch 13: 67659.625
Loss after epoch 14: 66321.375
Loss after epoch 15: 67882.25
Loss after epoch 16: 67352.125
Loss after epoch 17: 67269.875
Loss after epoch 18: 67082.625
Loss after epoch 19: 67428.25


(180246, 403120)

### 4 - Ensayar

In [15]:
vocabulario = list(w2v_model.wv.index_to_key)

In [16]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["julieta"], topn=10)

[('¡ay', 0.9874550104141235),
 ('dónde', 0.9758873581886292),
 ('mal', 0.9735255837440491),
 ('infame', 0.966683030128479),
 ('acuerdo', 0.9634206891059875),
 ('sangre', 0.9574002623558044),
 ('¡oh', 0.9561091065406799),
 ('¿te', 0.9560413956642151),
 ('viene', 0.9557097554206848),
 ('ciertamente', 0.9548540711402893)]

In [17]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["romeo"], topn=10)

[('muerto', 0.9612892270088196),
 ('teobaldo', 0.956717312335968),
 ('mató', 0.9565241932868958),
 ('vuelve', 0.9552223086357117),
 ('segun', 0.9509615898132324),
 ('igual', 0.9506431221961975),
 ('benvolio', 0.9494649767875671),
 ('tienes', 0.9489181637763977),
 ('habia', 0.9444577693939209),
 ('huye', 0.9440534114837646)]

In [18]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["capuleto"], topn=5)

[('su', 0.8364455103874207),
 ('tu', 0.8237165808677673),
 ('entran', 0.7916458249092102),
 ('de', 0.7807651162147522),
 ('murió', 0.7633351683616638)]

In [19]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["julieta"], topn=5)

[('ni', -0.47430524230003357),
 ('1', -0.5072617530822754),
 ('fray', -0.536050021648407),
 ('ojos', -0.6054343581199646),
 ('músico', -0.605552613735199)]

In [20]:
vector_romeo = w2v_model.wv.get_vector("romeo")
vector_julieta = w2v_model.wv.get_vector("julieta")
vector_montesco = w2v_model.wv.get_vector("montesco")
vector_capuleto = w2v_model.wv.get_vector("capuleto")

vector_resultado = vector_capuleto - vector_julieta + vector_romeo

In [21]:
w2v_model.wv.most_similar(vector_resultado)
# Acá esperaba obtener "montesco" como primera opción, pero aparece recién en la 7ma posición

[('capuleto', 0.9785167574882507),
 ('su', 0.8263970613479614),
 ('tu', 0.8174126148223877),
 ('de', 0.8054048418998718),
 ('entran', 0.765863299369812),
 ('murió', 0.7419558763504028),
 ('montesco', 0.7342888712882996),
 ('páris', 0.7310613393783569),
 ('tanto', 0.7308179140090942),
 ('romeo', 0.7272732853889465)]

In [22]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["montesco"], topn=5)

[('murió', 0.9579949975013733),
 ('¡ay', 0.9524285793304443),
 ('muerto', 0.9502136707305908),
 ('dónde', 0.9484080076217651),
 ('segun', 0.9462881088256836)]

In [31]:
# Similaridad de "romeo" y "julieta"
w2v_model.wv.similarity('romeo', 'julieta')

0.91236556

In [24]:
# Similaridad de "montesco" y "capuleto"
w2v_model.wv.similarity('montesco', 'capuleto')

0.74907213

### 5 - Visualizar agrupación de vectores

In [25]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [26]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=105
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
#fig.show(renderer="colab") # esto para plotly en colab
fig.show()

# Se puede observar en el primer cuadrante (arriba a la derecha) que están las palabras "capuleto" y "montesco".
# Sin embargo, "romeo" está también en el primer cuadrante, mientras que "julieta" está abajo de todo al centro.

In [27]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
#fig.show(renderer="colab") # esto para plotly en colab
fig.show()

In [28]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

### Alumno

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

# Conclusiones

Se crearon vectores de palabras con Gensin aplicados al modelo Skip-gram.
Se evaluó el texto de Romeo y Julieta.
Se obtuvieron representaciones gráficas del modelo.
Se prestó especial interés en las palabras "romeo", "julieta", "montesco" y "capuleto", buscando encontrar una relación entre los nombres y los apellidos de los personajes.
Algunas conclusiones de lo probado son que las palabras "montesco" y "capuleto" se encuentran muy cercanas en el gráfico, pero no tan cerca en el valor de similaridad (~0,74).
Por el contrario, aunque las palabras "romeo" y "julieta" tienen una gran similitud (~0,91), se encuentran alejadas en el gráfico, lo que puede significar que se usan en contextos diferentes a lo largo del texto.
Estas separaciones pueden deberse a limitaciones del modelo y al tamaño del corpus.